# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
ee.Authenticate()

## Load Authentication

The above authentication step creates credentials that are stored on your local computer. Those credentials need to be loaded so that Earth Engine and Pydeck will work.

First run the standard Earth Engine initialization:

In [2]:
ee.Initialize()

A further token is necessary for use with Pydeck. Here we ping the Google OAuth2 API to retrieve that token.

In [3]:
credentials = ee.data.get_persistent_credentials()
url = 'https://www.googleapis.com/oauth2/v4/token'
data = {
    'client_id': credentials.client_id,
    'client_secret': credentials.client_secret,
    'refresh_token': credentials.refresh_token,
    'grant_type': 'refresh_token'
}
r = requests.post(url, data=data)
access_token = r.json()['access_token']

`access_token` now holds a token that you can pass to the Pydeck Earth Engine layer.

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object.

In [4]:
image = ee.Image('CGIAR/SRTM90_V4')

The `Image` object itself can't be passed as-is to Pydeck. To create an object that can be passed to Pydeck, call the `.serialize()` method:

In [5]:
ee_object = image.serialize()

Since the `EarthEngineLayer` isn't built in to Deck.gl, we need to tell Pydeck to load a custom layer extension. Pass a URL to the built `earthengine-layers` bundle.

In [6]:
url = 'https://cdn.jsdelivr.net/gh/kylebarron/earthengine-layers@dev/modules/earthengine-layers/dist/bundle.js'
pdk.settings.custom_libraries = [
    {
        "libraryName": "EarthEngineLayerLibrary",
        "resourceUri": url,
    }
]

Now we're ready to create the Pydeck layer. Note that you must include the `token` and serialized `ee_object` here. Currently in Pydeck, you must wrap each string in quotes, so that Pydeck doesn't interpret the string as a function. Also, the character used to wrap the string must not appear in the string, so please use a literal `'`.

In [7]:
ee_layer = pdk.Layer(
    "EarthEngineLayer",
    None,
    token=f"'{access_token}'",
    ee_object=f"'{ee_object}'",
    vis_params={"min": 0, "max": 255}
)

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)
r = pdk.Deck(
    layers=[ee_layer], 
    initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…